In [23]:
# Import pandas for data manipulation and analysis
import pandas as pd


# 1) Dataset exploration

###  Load the row data as DataFrame and display the first 5 Rows

In [24]:
df = pd.read_csv("../data/row_data.csv")
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


### Dataset shape
The dataset contains **10,000 rows** and **8 columns**.
Each row represents a transaction, and the columns describe attributes such as item, quantity, price, payment method, and transaction date.

In [25]:
df.shape

(10000, 8)

### Dataset information
The table below shows the **structure of the dataset**, including the column names, the number of non-null values and the data types.
This helps to identify which columns contain **missing data** and whether data types need to be converted for further analysis.


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


### Statistical summary
The table below shows for each column the number of non-null values, the number of unique values, the most frequent value and how many times it appears (frequency)

In [27]:
df.describe()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_1961373,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


# 2) Data quality check

### Detecting **missing values**
The table below shows us how many missing values in each column

In [28]:
df.isnull().sum()

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

### Detecting duplicate records
The result shows that there are **no duplicate rows** in the dataset.
This means we can proceed without the need to remove any duplicates.

In [29]:
df.duplicated().sum()

0

### Distribution of categorical columns
Since we have three **categorical columns** (*Item*, *Payment Method*, *Location*),
we should inspect them to see which unique values they contain and whether they include
any inconsistent entries or placeholder values (e.g., *UNKNOWN*).


The table below shows that the **Item** column contains 344 entries with the value *UNKNOWN* and 292 entries with the value *ERROR*.

In [30]:
df["Item"].value_counts()

Item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
UNKNOWN      344
ERROR        292
Name: count, dtype: int64

The table below shows that the **Payment Method** column contains 293 entries with the value *UNKNOWN* and 306 entries with the value *ERROR*.

In [31]:
df["Payment Method"].value_counts()

Payment Method
Digital Wallet    2291
Credit Card       2273
Cash              2258
ERROR              306
UNKNOWN            293
Name: count, dtype: int64

The table below shows that the **Location** column contains 338 entries with the value *UNKNOWN* and 358 entries with the value *ERROR*.

In [32]:
df["Location"].value_counts()

Location
Takeaway    3022
In-store    3017
ERROR        358
UNKNOWN      338
Name: count, dtype: int64

# 3) Data Cleaning

### Cleaning the *Item* column
The column *Item* contains invalid placeholder values such as **UNKNOWN** and **ERROR**.
Since these values do not represent real products, they should be removed from the dataset.
We will filter out these rows to keep only valid items.

In [33]:
# Remove rows where Item is UNKNOWN or ERROR
df = df[~df["Item"].isin(["UNKNOWN", "ERROR"])]

# Verify the result
df["Item"].value_counts()

Item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
Name: count, dtype: int64

### Cleaning the *Payment Method* column

The column *Payment Method* contains invalid entries such as **UNKNOWN** and **Error**.
To avoid losing too many rows, we replace these values with **Cash**
(as it is the most common payment method in the dataset).
This ensures that all rows remain usable for analysis.


In [34]:
# Replace UNKNOWN values in Payment Method with 'Cash'
df["Payment Method"] = df["Payment Method"].replace(["UNKNOWN","ERROR"], "Cash")

# Verify the result
df["Payment Method"].value_counts()

Payment Method
Cash              2681
Digital Wallet    2147
Credit Card       2124
Name: count, dtype: int64

### Cleaning the *Location* column

The column *Location* contains invalid placeholder values such as **UNKNOWN** and **ERROR**.
Since *In-store* is the most frequent and most reliable category, we standardize all invalid values to **In-store**.
This ensures consistency in the dataset while keeping all rows usable.


In [35]:
# Replace all invalid values with "In-store"
df["Location"] = df["Location"].replace(["UNKNOWN", "ERROR"], "In-store")

# Verify the result
df["Location"].value_counts()


Location
In-store    3485
Takeaway    2830
Name: count, dtype: int64

### Cleaning numerical columns

The numerical columns (*Quantity*, *Price Per Unit*, *Total Spent*) may contain invalid values such as **ERROR** or missing entries.
To ensure data consistency:
- We convert all values to numeric format (invalid strings become NaN).
- We recalculate *Total Spent* as **Quantity × Price Per Unit** wherever it is missing or invalid.
This guarantees that all numerical columns contain only valid numbers.


In [36]:
# Convert Quantity and Price Per Unit to numeric (invalid entries → NaN)
df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")
df["Price Per Unit"] = pd.to_numeric(df["Price Per Unit"], errors="coerce")
df["Total Spent"] = pd.to_numeric(df["Total Spent"], errors="coerce")

# Recalculate Total Spent where missing or invalid
df["Total Spent"] = df["Quantity"] * df["Price Per Unit"]

# Verify the result
df[["Quantity", "Price Per Unit", "Total Spent"]].head()

,Quantity,Price Per Unit,Total Spent
0,2.0,2.0,4.0
1,4.0,3.0,12.0
2,4.0,1.0,4.0
3,2.0,5.0,10.0
4,2.0,2.0,4.0


### Cleaning the *Transaction Date* column

The column *Transaction Date* should contain only valid dates.
To ensure consistency:
- We convert the column to **datetime format**.
- Invalid entries are converted to **NaT** (Not a Time).
- This allows us to perform time-based analysis (e.g., sales per month).


In [37]:
# Convert Transaction Date to datetime (invalid entries → NaT)
df["Transaction Date"] = pd.to_datetime(df["Transaction Date"], errors="coerce")

# Verify the result
df["Transaction Date"].head()

0   2023-09-08
1   2023-05-16
2   2023-07-19
3   2023-04-27
4   2023-06-11
Name: Transaction Date, dtype: datetime64[ns]

### Handling missing values


In [38]:
# Display the number of the missing values per column
df.isnull().sum()

Transaction ID         0
Item                 333
Quantity             438
Price Per Unit       502
Total Spent          924
Payment Method      2412
Location            3049
Transaction Date     436
dtype: int64


The columns *Item* and *Transaction Date* each contain around **3–4% missing values**.
Since these features are critical for the analysis (we cannot interpret a transaction without knowing the product or the date),
we remove these rows. The data loss is minimal and does not affect the overall dataset quality.


In [39]:
# Drop rows with missing Item or Transaction Date
df = df.dropna(subset=["Item", "Transaction Date"])

### Handling missing values in *Quantity* and *Price Per Unit*

The columns *Quantity* and *Price Per Unit* contain around 4–5% missing values.
Instead of dropping these rows, we replace the missing values with the **median** of each column.
This approach preserves most of the data while avoiding distortion from outliers.


In [40]:
# Fill missing numerical values with the median
df["Quantity"] = df["Quantity"].fillna(df["Quantity"].median())
df["Price Per Unit"] = df["Price Per Unit"].fillna(df["Price Per Unit"].median())

### Fixing missing values in *Total Spent*

The column *Total Spent* contains missing values.
Since this column should always equal **Quantity × Price Per Unit**,
we recalculate it to replace NaN values with the correct amounts.

In [41]:
# Recalculate Total Spent where missing or invalid
df["Total Spent"] = df["Quantity"] * df["Price Per Unit"]

### Fixing missing values in *Payment Method* and *Location*

Missing values in categorical columns are replaced with the **most frequent value (mode)**.
This preserves all rows and keeps the categories consistent.


In [42]:
# Fill missing Payment Method and Location with mode
df["Payment Method"] = df["Payment Method"].fillna(df["Payment Method"].mode()[0])
df["Location"] = df["Location"].fillna(df["Location"].mode()[0])


### Final check for missing values


In [43]:
df.isnull().sum()

Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

### Export cleaned dataset
We save the cleaned DataFrame as a CSV for reuse in the EDA notebook.


In [44]:
df.to_csv("../data/cleaned_cafe_sales.csv", index=False)


# Data cleaning summary

All missing and invalid values have been handled successfully:
- *Item* and *Transaction Date*: rows with missing values removed (~3–4%).
- *Quantity* and *Price Per Unit*: recalculated where possible, otherwise filled with median.
- *Total Spent*: fully recalculated as Quantity × Price Per Unit.
- *Payment Method* and *Location*: missing values replaced with the most frequent value.

As a result, the dataset is now **clean, consistent, and ready for analysis**.
